# Searching for best hyperparameters set

# AdaBoost optimization

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import cross_val_score
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_parallel_coordinate
import os
import pickle

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
train_df = pd.read_feather('./data/train_processed.ftr')
val_df = pd.read_feather('./data/val_processed.ftr')
cols = list(train_df.columns)

In [ ]:
cols

## Prepare AdaBoost model

In [ ]:
train_x = train_df[cols[:-1]].to_numpy()
train_y = train_df[cols[-1]].to_numpy()
val_x = val_df[cols[:-1]].to_numpy()
val_y = val_df[cols[-1]].to_numpy()

In [ ]:
class ModelTester:
    
    def __init__(
        self,
        params: dict,
        train_x: np.array,
        train_y: np.array,
        val_x: np.array,
        val_y: np.array
    ):
        self.params = params
        self.train_x = train_x
        self.train_y = train_y
        self.val_x = val_x
        self.val_y = val_y
        
        self.model = self.train_model()
        self.y_pred_bin = self.get_bin_predictions()
            
    def train_model(self):
        # 1 - Run model
        tree = DecisionTreeClassifier(
            min_samples_split=self.params['min_samples_split'],
            max_depth=self.params['max_depth'],
            min_impurity_decrease=self.params['min_impurity_decrease'],
            criterion=self.params['criterion']
        )
        model = AdaBoostClassifier(
            base_estimator=tree,
            n_estimators=self.params['n_estimators'],
            learning_rate=self.params['learning_rate'],
            algorithm=self.params['algorithm'],
        )
        model.fit(self.train_x, self.train_y)
        return model
    
    def get_bin_predictions(self):
        # 2 - Get predictions
        y_pred = self.model.predict(val_x)
        y_pred_bin = [1 if y else 0 for y in y_pred]
        
        return y_pred_bin
    
    def get_accuracy(self):
        return round(accuracy_score(self.val_y, self.y_pred_bin), 3)
    
    def get_class_report(self):
        return classification_report(self.val_y, self.y_pred_bin)    

## Optuna research

In [ ]:
def objective(trial):
    # 0 - Prepare params
    n_estimators = trial.suggest_int("n_estimators", 10, 200)
    algorithm = trial.suggest_categorical('algorithm', ['SAMME', 'SAMME.R'])
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.5)
    max_depth = trial.suggest_int('max_depth', 1, 100)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 100)
    min_impurity_decrease = trial.suggest_float('min_impurity_decrease', 0, 0.0001)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'algorithm': algorithm,
        'max_depth': max_depth,
        'min_samples_split': min_samples_split,
        'min_impurity_decrease': min_impurity_decrease,
        'criterion': criterion
    }
    
    # 1 - Model
    model_tester = ModelTester(params, train_x, train_y, val_x, val_y)
    acc = model_tester.get_accuracy()
    error = 1 - acc
    
    return error

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=1000)

### Show results

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
key_params = ['algorithm', 'learning_rate', 'n_estimators']
plot_slice(study, params = key_params).show()

In [ ]:
study.best_params

In [ ]:
best_params = {
    'n_estimators': 106,
    'algorithm': 'SAMME.R',
    'learning_rate': 0.4827054897488229,
    'max_depth': 11,
    'min_samples_split': 1,
    'min_impurity_decrease': 7.200517833654635e-05,
    'criterion': 'gini'
}

## Create model with best params

In [ ]:
train_x = train_df[cols[:-1]].to_numpy()
train_y = train_df[cols[-1]].to_numpy()
val_x = val_df[cols[:-1]].to_numpy()
val_y = val_df[cols[-1]].to_numpy()

In [ ]:
params = best_params
tree = DecisionTreeClassifier(
#     min_samples_split=1,
    max_depth=11,
    min_impurity_decrease=7.200517833654635e-05,
    criterion='gini'
)
model = AdaBoostClassifier(
    base_estimator=tree,
    n_estimators=106,
    learning_rate=0.4827054897488229,
    algorithm='SAMME.R'
)

In [ ]:
X = pd.concat([train_df[cols[:-1]], val_df[cols[:-1]]])
y = pd.concat([train_df[cols[-1]], val_df[cols[-1]]])

In [ ]:
cv_scores = cross_val_score(model, X, y, cv=10)
print(f'Mean cv (k=10) accuracy score = {round(np.mean(cv_scores), 3)}')

## Feature selection

In [ ]:
model.fit(train_x, train_y)

In [ ]:
imp_df = pd.DataFrame({
    'feature': cols[:-1],
    'imp': model.feature_importances_
}).sort_values('imp', ascending=False)
best_features = imp_df['feature'].to_list()
imp_df

In [ ]:
temp_cols = []
cv_list = []

for feature in best_features:
    temp_cols.append(feature)
    X = pd.concat([train_df[temp_cols], val_df[temp_cols]]).to_numpy()
    cv_score = cross_val_score(model, X, y, cv=10)
    cv_list.append(cv_score)

In [ ]:
means = []
for scores in cv_list:
    means.append(round(np.mean(scores), 3))

i_features = 0
for i in range(len(means)):
    if means[i] == max(means):
        i_features = i
        print(i_features)
        break

In [ ]:
best_features[:i_features+1]

## Make final model

In [ ]:
train_x = train_df[best_features[:i_features+1]].to_numpy()
val_x = val_df[best_features[:i_features+1]].to_numpy()

In [ ]:
params = best_params
tree = DecisionTreeClassifier(
#     min_samples_split=1,
    max_depth=11,
    min_impurity_decrease=7.200517833654635e-05,
    criterion='gini'
)
model = AdaBoostClassifier(
    base_estimator=tree,
    n_estimators=106,
    learning_rate=0.4827054897488229,
    algorithm='SAMME.R'
)

In [ ]:
model.fit(train_x, train_y)
pred_val = model.predict_proba(val_x)
pred_train = model.predict_proba(train_x)

In [ ]:
model_dict = {
    'model': model,
    'features': best_features
}

filepath = os.path.join('models', 'adaboost.pickle')
with open(filepath, 'wb') as file:
    pickle.dump(model_dict, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.fit(train_x, train_y)
pred_val = model.predict_proba(val_x)
pred_train = model.predict_proba(train_x)

In [ ]:
train_proba_df = pd.DataFrame(pred_train)
train_proba_df.columns = ['0', 'ada_boost']
train_proba_df['y'] = train_y.astype(int)
train_proba_df.drop(columns='0', inplace=True)
train_proba_df

In [ ]:
val_proba_df = pd.DataFrame(pred_val)
val_proba_df.columns = ['0', 'ada_boost']
val_proba_df['y'] = val_y.astype(int)
val_proba_df.drop(columns='0', inplace=True)
val_proba_df

In [ ]:
train_proba_df.to_csv('./data/ensemble_train_df.csv')
val_proba_df.to_csv('./data/ensemble_val_df.csv')

In [ ]:
filepath = os.path.join('models', 'adaboost.pickle')
pickle.dump(model, open(filepath, 'wb'))